## AI Finance Project - Loda Enrico

In [1]:
!pip install yfinance

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.0 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.0 MB 932.9 kB/s eta 0:00:03
     ---------- ----------------------------- 0.8/3.0 MB 960.2 kB/s eta 0:00:03
     ------------- -------------------------- 1.0/3.0 MB 1.0 MB/s eta 0:00:02
     ----------------- ---------------------- 1.3/3.0 MB 987.4 kB/s eta 0:00:02
     ----------------- ---------------------- 1.3/3.0 MB 987.4 kB/s eta 0:00:02
     -------------------- ------------------- 1.6/3.0 MB 976.0 kB/s eta 0:00:02
     ------------------------ --------------- 1.8/3.0 MB 987.4 kB/s eta 0:00:02
     --------------------------- ------------ 2.1/3.0 MB 1.0 MB/s eta 0:00:01
     ---------------------------------- ----- 2.6/3.0 MB 1.1 MB/s eta 0:00:01
  


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# import
import pandas as pd
import yfinance as yf
import numpy as np
import torch

We want to calculate the daily simple rate of return of our assets, in order to do so we

\

**Daily simple rate of return :**
$
R_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_t}{P_{t-1}} - 1
$

In [ ]:
df = yf.download(["SPY","NVDA"], start="2010-01-01", end="2025-01-01")
spy = df.xs('SPY', axis=1, level=1)
nvda = df.xs('NVDA', axis=1, level=1)

prices = df["Close"]
returns = (prices / prices.shift(1)) - 1
returns = returns.dropna()
SPY_r = returns['SPY']
NVDA_r = returns['NVDA']

[*********************100%***********************]  2 of 2 completed


In [ ]:
df

Price            Close                    High                     Low  \
Ticker            NVDA         SPY        NVDA         SPY        NVDA   
Date                                                                     
2010-01-04    0.423884   85.768425    0.426864   85.813831    0.415172   
2010-01-05    0.430073   85.995499    0.434658   86.033341    0.422279   
2010-01-06    0.432824   86.056046    0.433741   86.267949    0.425718   
2010-01-07    0.424342   86.419273    0.432366   86.525225    0.421133   
2010-01-08    0.425259   86.706863    0.428239   86.744706    0.418382   
...                ...         ...         ...         ...         ...   
2024-12-24  140.207108  599.496582  141.886946  599.536501  138.637245   
2024-12-26  139.917130  599.536499  140.837058  600.673034  137.717335   
2024-12-27  136.997391  593.225464  139.007216  595.987176  134.697615   
2024-12-30  137.477356  586.455811  140.257099  589.965273  134.007674   
2024-12-31  134.277649  584.322266  138.057315  588.868587  133.817700   

Price                         Open                 Volume             
Ticker             SPY        NVDA         SPY       NVDA        SPY  
Date                                                                  
2010-01-04   84.391045    0.424342   85.041895  800204000  118944600  
2010-01-05   85.405193    0.422279   85.715485  728648000  111579900  
2010-01-06   85.844142    0.429844   85.912251  649168000  116074400  
2010-01-07   85.654901    0.430532   85.897078  547792000  131091100  
2010-01-08   86.018176    0.420903   86.192237  478168000  126402800  
...                ...         ...         ...        ...        ...  
2024-12-24  593.684050  139.987127  594.272307  105157000   33160100  
2024-12-26  596.286267  139.687155  597.701991  116205600   41219100  
2024-12-27  588.988210  138.537258  595.747844  170582600   64969300  
2024-12-30  582.657240  134.817597  586.126844  167734700   56578800  
2024-12-31  582.667210  138.017311  588.140735  155659200   57052700  

[3774 rows x 10 columns]

In [ ]:
returns[:]

Ticker,NVDA,SPY
Date,,
2010-01-05,0.014603,0.002648
2010-01-06,0.006396,0.000704
2010-01-07,-0.019598,0.004221
2010-01-08,0.002161,0.003328
2010-01-11,-0.014016,0.001397
...,...,...
2024-12-24,0.003938,0.011115
2024-12-26,-0.002068,0.000067
2024-12-27,-0.020868,-0.010527


#dataset for training
The features, for a total of 24, are the following:

*   $\mu_1$, $\mu_2$ of $r_1$, $r_2$ calculated on the last 10 lags
*   $[r_1^t,\dots ,r_1^{t-10}]$, $[r_2^t,\dots ,r_2^{t-10}]$

The target is the values of my covariance matrix:
* $\Sigma_{t+1}$

In [ ]:
rNVDA_series = returns['NVDA']
rSPY_series = returns['SPY']
df = pd.DataFrame()
df['rNVDA_today'] = rNVDA_series
df['rSPY_today'] = rSPY_series

for i in range(1, 11):
    df[f'rNVDA_t-{i}'] = rNVDA_series.shift(i)
    df[f'rSPY_t-{i}'] = rSPY_series.shift(i)

df.head(11)

,rNVDA_today,rSPY_today,rNVDA_t-1,rSPY_t-1,rNVDA_t-2,rSPY_t-2,rNVDA_t-3,rSPY_t-3,rNVDA_t-4,rSPY_t-4,...,rNVDA_t-6,rSPY_t-6,rNVDA_t-7,rSPY_t-7,rNVDA_t-8,rSPY_t-8,rNVDA_t-9,rSPY_t-9,rNVDA_t-10,rSPY_t-10
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.014603,0.002648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.006396,0.000704,0.014603,0.002648,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,-0.019598,0.004221,0.006396,0.000704,0.014603,0.002648,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.002161,0.003328,-0.019598,0.004221,0.006396,0.000704,0.014603,0.002648,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,-0.014016,0.001397,0.002161,0.003328,-0.019598,0.004221,0.006396,0.000704,0.014603,0.002648,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-12,-0.033898,-0.009326,-0.014016,0.001397,0.002161,0.003328,-0.019598,0.004221,0.006396,0.000704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-13,0.013582,0.008446,-0.033898,-0.009326,-0.014016,0.001397,0.002161,0.003328,-0.019598,0.004221,...,0.014603,0.002648,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-14,-0.015633,0.002705,0.013582,0.008446,-0.033898,-0.009326,-0.014016,0.001397,0.002161,0.003328,...,0.006396,0.000704,0.014603,0.002648,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-15,-0.029495,-0.011224,-0.015633,0.002705,0.013582,0.008446,-0.033898,-0.009326,-0.014016,0.001397,...,-0.019598,0.004221,0.006396,0.000704,0.014603,0.002648,NaN,NaN,NaN,NaN


In [ ]:
cols = [f'rNVDA_t-{i}' for i in range(1, 11)]
df['mean_NVDA'] = df[cols].mean(axis=1)
cols.append('rNVDA_today')
df['var_NVDA'] = df[cols].var(axis=1)

cols = [f'rSPY_t-{i}' for i in range(1, 11)]
df['mean_SPY'] = df[cols].mean(axis=1)
cols.append('rSPY_today')
df['var_SPY'] = df[cols].var(axis=1)
df = df.dropna()
df#["r2_t-10"]

,rNVDA_today,rSPY_today,rNVDA_t-1,rSPY_t-1,rNVDA_t-2,rSPY_t-2,rNVDA_t-3,rSPY_t-3,rNVDA_t-4,rSPY_t-4,...,rNVDA_t-8,rSPY_t-8,rNVDA_t-9,rSPY_t-9,rNVDA_t-10,rSPY_t-10,mean_NVDA,var_NVDA,mean_SPY,var_SPY
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-20,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,0.013582,0.008446,...,-0.019598,0.004221,0.006396,0.000704,0.014603,0.002648,-0.005720,0.000331,0.001539,0.000059
2010-01-21,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,...,0.002161,0.003328,-0.019598,0.004221,0.006396,0.000704,-0.007581,0.000296,0.000258,0.000093
2010-01-22,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,...,-0.014016,0.001397,0.002161,0.003328,-0.019598,0.004221,-0.010007,0.000327,-0.001736,0.000131
2010-01-25,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,...,-0.033898,-0.009326,-0.014016,0.001397,0.002161,0.003328,-0.011507,0.000395,-0.004387,0.000132
2010-01-26,-0.031661,-0.004191,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,...,0.013582,0.008446,-0.033898,-0.009326,-0.014016,0.001397,-0.010022,0.000424,-0.004207,0.000127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,0.003938,0.011115,0.036897,0.005988,0.030762,0.012011,0.013730,-0.000307,-0.011351,-0.029803,...,-0.014141,-0.005153,0.031391,0.007731,-0.026943,-0.003109,0.000889,0.000547,-0.001269,0.000132
2024-12-26,-0.002068,0.000067,0.003938,0.011115,0.036897,0.005988,0.030762,0.012011,0.013730,-0.000307,...,-0.022499,-0.000199,-0.014141,-0.005153,0.031391,0.007731,0.003977,0.000464,0.000153,0.000131
2024-12-27,-0.020868,-0.010527,-0.002068,0.000067,0.003938,0.011115,0.036897,0.005988,0.030762,0.012011,...,-0.016760,0.004270,-0.022499,-0.000199,-0.014141,-0.005153,0.000631,0.000420,-0.000613,0.000134


In [ ]:
def compute_covariance_row(row):
    r1 = [row[f'rNVDA_t-{i}'] for i in range(1, 11)]
    r1.append(row['rNVDA_today'])

    r2 = [row[f'rSPY_t-{i}'] for i in range(1, 11)]
    r2.append(row['rSPY_today'])

    return np.cov(r1, r2)[0, 1]

df['cov'] = df.apply(compute_covariance_row, axis=1)
df

,rNVDA_today,rSPY_today,rNVDA_t-1,rSPY_t-1,rNVDA_t-2,rSPY_t-2,rNVDA_t-3,rSPY_t-3,rNVDA_t-4,rSPY_t-4,...,rSPY_t-8,rNVDA_t-9,rSPY_t-9,rNVDA_t-10,rSPY_t-10,mean_NVDA,var_NVDA,mean_SPY,var_SPY,cov
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-20,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,0.013582,0.008446,...,0.004221,0.006396,0.000704,0.014603,0.002648,-0.005720,0.000331,0.001539,0.000059,0.000097
2010-01-21,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,...,0.003328,-0.019598,0.004221,0.006396,0.000704,-0.007581,0.000296,0.000258,0.000093,0.000111
2010-01-22,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,...,0.001397,0.002161,0.003328,-0.019598,0.004221,-0.010007,0.000327,-0.001736,0.000131,0.000153
2010-01-25,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,...,-0.009326,-0.014016,0.001397,0.002161,0.003328,-0.011507,0.000395,-0.004387,0.000132,0.000184
2010-01-26,-0.031661,-0.004191,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,...,0.008446,-0.033898,-0.009326,-0.014016,0.001397,-0.010022,0.000424,-0.004207,0.000127,0.000175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,0.003938,0.011115,0.036897,0.005988,0.030762,0.012011,0.013730,-0.000307,-0.011351,-0.029803,...,-0.005153,0.031391,0.007731,-0.026943,-0.003109,0.000889,0.000547,-0.001269,0.000132,0.000135
2024-12-26,-0.002068,0.000067,0.003938,0.011115,0.036897,0.005988,0.030762,0.012011,0.013730,-0.000307,...,-0.000199,-0.014141,-0.005153,0.031391,0.007731,0.003977,0.000464,0.000153,0.000131,0.000126
2024-12-27,-0.020868,-0.010527,-0.002068,0.000067,0.003938,0.011115,0.036897,0.005988,0.030762,0.012011,...,0.004270,-0.022499,-0.000199,-0.014141,-0.005153,0.000631,0.000420,-0.000613,0.000134,0.000122


In [ ]:
df.drop(['rNVDA_today', 'rSPY_today'], axis=1, inplace=True)
df

,rNVDA_t-1,rSPY_t-1,rNVDA_t-2,rSPY_t-2,rNVDA_t-3,rSPY_t-3,rNVDA_t-4,rSPY_t-4,rNVDA_t-5,rSPY_t-5,...,rSPY_t-8,rNVDA_t-9,rSPY_t-9,rNVDA_t-10,rSPY_t-10,mean_NVDA,var_NVDA,mean_SPY,var_SPY,cov
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-20,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,0.013582,0.008446,-0.033898,-0.009326,...,0.004221,0.006396,0.000704,0.014603,0.002648,-0.005720,0.000331,0.001539,0.000059,0.000097
2010-01-21,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,0.013582,0.008446,...,0.003328,-0.019598,0.004221,0.006396,0.000704,-0.007581,0.000296,0.000258,0.000093,0.000111
2010-01-22,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,...,0.001397,0.002161,0.003328,-0.019598,0.004221,-0.010007,0.000327,-0.001736,0.000131,0.000153
2010-01-25,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,...,-0.009326,-0.014016,0.001397,0.002161,0.003328,-0.011507,0.000395,-0.004387,0.000132,0.000184
2010-01-26,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,...,0.008446,-0.033898,-0.009326,-0.014016,0.001397,-0.010022,0.000424,-0.004207,0.000127,0.000175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,0.036897,0.005988,0.030762,0.012011,0.013730,-0.000307,-0.011351,-0.029803,-0.012197,-0.004120,...,-0.005153,0.031391,0.007731,-0.026943,-0.003109,0.000889,0.000547,-0.001269,0.000132,0.000135
2024-12-26,0.003938,0.011115,0.036897,0.005988,0.030762,0.012011,0.013730,-0.000307,-0.011351,-0.029803,...,-0.000199,-0.014141,-0.005153,0.031391,0.007731,0.003977,0.000464,0.000153,0.000131,0.000126
2024-12-27,-0.002068,0.000067,0.003938,0.011115,0.036897,0.005988,0.030762,0.012011,0.013730,-0.000307,...,0.004270,-0.022499,-0.000199,-0.014141,-0.005153,0.000631,0.000420,-0.000613,0.000134,0.000122


In [ ]:
train_df = df.loc["2010-01-01":"2023-12-31"]
test_df  = df.loc["2024-01-01":]
print((len(train_df) + len(test_df)) == len(df))

True


In [ ]:
train_df

,rNVDA_t-1,rSPY_t-1,rNVDA_t-2,rSPY_t-2,rNVDA_t-3,rSPY_t-3,rNVDA_t-4,rSPY_t-4,rNVDA_t-5,rSPY_t-5,...,rSPY_t-8,rNVDA_t-9,rSPY_t-9,rNVDA_t-10,rSPY_t-10,mean_NVDA,var_NVDA,mean_SPY,var_SPY,cov
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-20,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,0.013582,0.008446,-0.033898,-0.009326,...,0.004221,0.006396,0.000704,0.014603,0.002648,-0.005720,0.000331,0.001539,0.000059,0.000097
2010-01-21,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,0.013582,0.008446,...,0.003328,-0.019598,0.004221,0.006396,0.000704,-0.007581,0.000296,0.000258,0.000093,0.000111
2010-01-22,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,-0.015633,0.002705,...,0.001397,0.002161,0.003328,-0.019598,0.004221,-0.010007,0.000327,-0.001736,0.000131,0.000153
2010-01-25,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,-0.029495,-0.011224,...,-0.009326,-0.014016,0.001397,0.002161,0.003328,-0.011507,0.000395,-0.004387,0.000132,0.000184
2010-01-26,0.017011,0.005128,-0.034604,-0.022292,-0.017857,-0.019229,-0.004016,-0.010168,0.018703,0.012496,...,0.008446,-0.033898,-0.009326,-0.014016,0.001397,-0.010022,0.000424,-0.004207,0.000127,0.000175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,0.018270,0.009482,-0.030098,-0.013857,-0.009445,0.006081,0.024279,0.005625,0.011169,-0.001647,...,0.004567,-0.018503,0.003889,0.019530,0.004299,0.005178,0.000316,0.003544,0.000048,0.000073
2023-12-26,-0.003266,0.002010,0.018270,0.009482,-0.030098,-0.013857,-0.009445,0.006081,0.024279,0.005625,...,0.013790,0.022090,0.004567,-0.018503,0.003889,0.002899,0.000295,0.003315,0.000048,0.000072
2023-12-27,0.009195,0.004223,-0.003266,0.002010,0.018270,0.009482,-0.030098,-0.013857,-0.009445,0.006081,...,0.003209,0.009044,0.013790,0.022090,0.004567,0.005669,0.000242,0.003348,0.000049,0.000074


In [ ]:
target_cols = ['var_NVDA', 'var_SPY', 'cov']
X_train = train_df.drop(columns=target_cols)
y_train = train_df[target_cols]

print(X_train.head())
print("-"*10)
y_train.head()

            rNVDA_t-1  rSPY_t-1  rNVDA_t-2  rSPY_t-2  rNVDA_t-3  rSPY_t-3  \
Date                                                                        
2010-01-20   0.018703  0.012496  -0.029495 -0.011224  -0.015633  0.002705   
2010-01-21  -0.004016 -0.010168   0.018703  0.012496  -0.029495 -0.011224   
2010-01-22  -0.017857 -0.019229  -0.004016 -0.010168   0.018703  0.012496   
2010-01-25  -0.034604 -0.022292  -0.017857 -0.019229  -0.004016 -0.010168   
2010-01-26   0.017011  0.005128  -0.034604 -0.022292  -0.017857 -0.019229   

            rNVDA_t-4  rSPY_t-4  rNVDA_t-5  rSPY_t-5  ...  rNVDA_t-7  \
Date                                                  ...              
2010-01-20   0.013582  0.008446  -0.033898 -0.009326  ...   0.002161   
2010-01-21  -0.015633  0.002705   0.013582  0.008446  ...  -0.014016   
2010-01-22  -0.029495 -0.011224  -0.015633  0.002705  ...  -0.033898   
2010-01-25   0.018703  0.012496  -0.029495 -0.011224  ...   0.013582   
2010-01-26  -0.004016 -0.010

,var_NVDA,var_SPY,cov
Date,,,
2010-01-20,0.000331,0.000059,0.000097
2010-01-21,0.000296,0.000093,0.000111
2010-01-22,0.000327,0.000131,0.000153
2010-01-25,0.000395,0.000132,0.000184
2010-01-26,0.000424,0.000127,0.000175


In [ ]:
mean_cols = ["mean_NVDA", "mean_SPY"]
X_train_lstm = X_train.drop(columns=mean_cols)
X_train_mean = X_train[mean_cols]